In [0]:
"""
CLIP Model Serving service for generating embeddings
Endpoint: clip-image-encoder
Dimension: 512 (ViT-B/32)
"""
import base64
import logging
import numpy as np
import os

logger = logging.getLogger(__name__)


class CLIPService:
    """Service for interacting with CLIP model serving endpoint"""
    
    def __init__(self):
        self.endpoint_name = "clip-image-encoder"
        self.workspace_host = os.getenv("DATABRICKS_HOST", "")
        if not self.workspace_host.startswith("http"):
            self.workspace_host = f"https://{self.workspace_host}"
        self.embedding_dim = 512  # CLIP ViT-B/32
        
    def _get_endpoint_url(self) -> str:
        """Construct the full endpoint URL"""
        return f"{self.workspace_host}/serving-endpoints/{self.endpoint_name}/invocations"
    
    def _get_auth_headers(self) -> dict:
        """Get authorization headers with fresh OAuth token"""
        from databricks.sdk import WorkspaceClient
        w = WorkspaceClient()
        token = w.config.oauth_token().access_token
        return {
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json"
        }
    
    async def get_image_embedding(self, image_bytes: bytes) -> np.ndarray:
        """
        Generate CLIP embedding for an image
        
        Args:
            image_bytes: Raw image bytes (JPEG, PNG, etc.)
            
        Returns:
            numpy array of shape (512,) - L2 normalized embedding
        """
        import aiohttp
        
        try:
            # Encode image to base64
            image_b64 = base64.b64encode(image_bytes).decode("utf-8")
            
            # Prepare payload in dataframe_records format (pyfunc model)
            payload = {
                "dataframe_records": [{"image": image_b64}]
            }
            
            logger.info(f"Calling CLIP endpoint for image embedding (size: {len(image_bytes)} bytes)")
            
            # Call Model Serving endpoint
            timeout = aiohttp.ClientTimeout(total=30)
            async with aiohttp.ClientSession(timeout=timeout) as session:
                async with session.post(
                    self._get_endpoint_url(),
                    json=payload,
                    headers=self._get_auth_headers()
                ) as response:
                    if response.status != 200:
                        error_text = await response.text()
                        logger.error(f"CLIP endpoint error {response.status}: {error_text}")
                        raise Exception(f"CLIP endpoint returned {response.status}")
                    
                    result = await response.json()
            
            # Parse response - handle different formats
            if isinstance(result, dict) and "predictions" in result:
                embedding = np.array(result["predictions"][0], dtype=np.float32)
            elif isinstance(result, list):
                embedding = np.array(result[0] if result else [], dtype=np.float32)
            else:
                embedding = np.array(result, dtype=np.float32)
            
            # Ensure L2 normalization (for cosine similarity on VS)
            norm = np.linalg.norm(embedding)
            if norm > 0:
                embedding = embedding / norm
            
            logger.info(f"Generated image embedding: shape={embedding.shape}, norm={np.linalg.norm(embedding):.4f}")
            
            return embedding
            
        except Exception as e:
            logger.error(f"Error generating image embedding: {type(e).__name__}: {e}")
            raise
    
    async def get_text_embedding(self, text: str) -> np.ndarray:
        """
        Generate CLIP embedding for text query
        
        Args:
            text: Text query (e.g., "red summer dress")
            
        Returns:
            numpy array of shape (512,) - L2 normalized embedding
        """
        import aiohttp
        
        try:
            # Prepare payload for text mode
            payload = {
                "dataframe_records": [{"text": text}]
            }
            
            logger.info(f"Calling CLIP endpoint for text embedding: '{text}'")
            
            # Call Model Serving endpoint
            timeout = aiohttp.ClientTimeout(total=30)
            async with aiohttp.ClientSession(timeout=timeout) as session:
                async with session.post(
                    self._get_endpoint_url(),
                    json=payload,
                    headers=self._get_auth_headers()
                ) as response:
                    if response.status != 200:
                        error_text = await response.text()
                        logger.error(f"CLIP endpoint error {response.status}: {error_text}")
                        raise Exception(f"CLIP endpoint returned {response.status}")
                    
                    result = await response.json()
            
            # Parse response
            if isinstance(result, dict) and "predictions" in result:
                embedding = np.array(result["predictions"][0], dtype=np.float32)
            elif isinstance(result, list):
                embedding = np.array(result[0] if result else [], dtype=np.float32)
            else:
                embedding = np.array(result, dtype=np.float32)
            
            # Ensure L2 normalization
            norm = np.linalg.norm(embedding)
            if norm > 0:
                embedding = embedding / norm
            
            logger.info(f"Generated text embedding: shape={embedding.shape}, norm={np.linalg.norm(embedding):.4f}")
            
            return embedding
            
        except Exception as e:
            logger.error(f"Error generating text embedding: {type(e).__name__}: {e}")
            raise


# Singleton instance
clip_service = CLIPService()

In [0]:
"""
Vector Search service for similarity queries
Endpoint: fashion_vector_search (4d329fc8-1924-4131-ace8-14b542f8c14b)
Index: main.fashion_demo.product_embeddings_index
"""
import logging
import numpy as np
from typing import List, Dict, Any, Optional
from databricks.vector_search.client import VectorSearchClient
import os

logger = logging.getLogger(__name__)


class VectorSearchService:
    """Service for Vector Search similarity queries"""
    
    def __init__(self):
        self.endpoint_name = "fashion_vector_search"
        self.endpoint_id = "4d329fc8-1924-4131-ace8-14b542f8c14b"
        self.index_name = "main.fashion_demo.product_embeddings_index"
        self.embedding_dim = 512
        self.workspace_host = os.getenv("DATABRICKS_HOST", "")
        if not self.workspace_host.startswith("http"):
            self.workspace_host = f"https://{self.workspace_host}"
        self._client = None
        self._index = None
    
    def _get_client(self) -> VectorSearchClient:
        """Get or create Vector Search client"""
        if self._client is None:
            # VectorSearchClient uses workspace auth automatically in Databricks Apps
            self._client = VectorSearchClient(
                workspace_url=self.workspace_host,
                disable_notice=True
            )
            logger.info(f"Created Vector Search client for {self.workspace_host}")
        return self._client
    
    def _get_index(self):
        """Get Vector Search index"""
        if self._index is None:
            client = self._get_client()
            self._index = client.get_index(self.index_name)
            logger.info(f"Connected to Vector Search index: {self.index_name}")
        return self._index
    
    async def similarity_search(
        self,
        query_vector: np.ndarray,
        num_results: int = 20,
        filters: Optional[Dict[str, Any]] = None
    ) -> List[Dict[str, Any]]:
        """
        Search for similar products using vector similarity
        
        Args:
            query_vector: Normalized embedding vector (512 dims)
            num_results: Number of results to return
            filters: Optional filters (e.g., {"price >= ": 50})
            
        Returns:
            List of product dictionaries with similarity scores
        """
        try:
            # Ensure vector is normalized and correct shape
            if query_vector.shape != (self.embedding_dim,):
                raise ValueError(f"Expected vector shape ({self.embedding_dim},), got {query_vector.shape}")
            
            # Ensure L2 normalization for cosine-like similarity
            norm = np.linalg.norm(query_vector)
            if norm > 0:
                query_vector = query_vector / norm
            
            logger.info(f"Vector Search query: dim={query_vector.shape[0]}, norm={np.linalg.norm(query_vector):.4f}, filters={filters}")
            
            # Get index and perform similarity search
            index = self._get_index()
            
            # Columns to return from the index
            columns = [
                "product_id",
                "product_display_name", 
                "master_category",
                "sub_category",
                "article_type",
                "base_color",
                "price",
                "image_path",
                "gender",
                "season",
                "usage",
                "year"
            ]
            
            # Perform similarity search
            # Note: Vector Search SDK is synchronous, wrap in executor
            import asyncio
            loop = asyncio.get_event_loop()
            
            def do_search():
                return index.similarity_search(
                    query_vector=query_vector.tolist(),
                    columns=columns,
                    num_results=num_results,
                    filters=filters
                )
            
            results = await loop.run_in_executor(None, do_search)
            
            # Parse results
            if "result" in results and "data_array" in results["result"]:
                data_array = results["result"]["data_array"]
                logger.info(f"Vector Search returned {len(data_array)} results")
                
                # Convert to list of dicts
                products = []
                for row in data_array:
                    product = dict(zip(columns, row))
                    products.append(product)
                
                return products
            else:
                logger.warning(f"Unexpected Vector Search response format: {results}")
                return []
                
        except Exception as e:
            logger.error(f"Vector Search error: {type(e).__name__}: {e}")
            raise


# Singleton instance
vector_search_service = VectorSearchService()

In [0]:
"""
Search API routes with CLIP + Vector Search integration
"""
from fastapi import APIRouter, HTTPException, UploadFile, File, Form, Depends
from sqlalchemy.ext.asyncio import AsyncSession
from typing import Optional
from models.schemas import SearchRequest, SearchResponse, ProductDetail
from repositories.lakebase import LakebaseRepository
from core.database import get_async_db
import numpy as np
import os
import logging

logger = logging.getLogger(__name__)

router = APIRouter(prefix="/search", tags=["search"])

# Get workspace host for constructing Files API URLs
WORKSPACE_HOST = os.getenv("DATABRICKS_HOST", "")
if WORKSPACE_HOST and not WORKSPACE_HOST.startswith("http"):
    WORKSPACE_HOST = f"https://{WORKSPACE_HOST}"


def get_image_url(product_id: int) -> str:
    """
    Construct direct Files API URL for product image
    """
    return f"{WORKSPACE_HOST}/ajax-api/2.0/fs/files/Volumes/main/fashion_demo/raw_data/images/{product_id}.jpg"


@router.post("/text", response_model=SearchResponse)
async def search_by_text(
    request: SearchRequest,
    db: AsyncSession = Depends(get_async_db)
):
    """
    Semantic text search using CLIP embeddings + Vector Search
    """
    try:
        from services.clip_service import clip_service
        from services.vector_search_service import vector_search_service
        
        logger.info(f"Text search request: '{request.query}' (limit={request.limit})")
        
        # Generate text embedding using CLIP
        text_embedding = await clip_service.get_text_embedding(request.query)
        
        # Search for similar products using Vector Search
        products_data = await vector_search_service.similarity_search(
            query_vector=text_embedding,
            num_results=request.limit,
            filters=request.filters
        )
        
        # Convert to ProductDetail
        products = []
        for p in products_data:
            product = ProductDetail(**p)
            product.image_url = get_image_url(int(product.product_id))
            # Similarity score comes from Vector Search
            product.similarity_score = p.get("score", 0.85)
            products.append(product)
        
        logger.info(f"Text search returned {len(products)} results")
        
        return SearchResponse(
            products=products,
            query=request.query,
            search_type="text",
            user_id=request.user_id
        )
        
    except Exception as e:
        logger.error(f"Text search error: {type(e).__name__}: {e}")
        # Fallback to basic text search if Vector Search fails
        logger.warning("Falling back to basic text search")
        repo = LakebaseRepository(db)
        products_data = await repo.search_products_by_text(
            query=request.query,
            limit=request.limit
        )
        
        products = []
        for p in products_data:
            product = ProductDetail(**p)
            product.image_url = get_image_url(int(product.product_id))
            product.similarity_score = 0.75
            products.append(product)
        
        return SearchResponse(
            products=products,
            query=request.query,
            search_type="text",
            user_id=request.user_id
        )


@router.post("/image", response_model=SearchResponse)
async def search_by_image(
    image: UploadFile = File(...),
    user_id: Optional[str] = Form(None),
    limit: int = Form(20),
    db: AsyncSession = Depends(get_async_db)
):
    """
    Visual search using CLIP image embeddings + Vector Search
    """
    try:
        from services.clip_service import clip_service
        from services.vector_search_service import vector_search_service
        
        logger.info(f"Image search request: {image.filename} (limit={limit})")
        
        # Read uploaded image
        image_bytes = await image.read()
        
        # Generate image embedding using CLIP
        image_embedding = await clip_service.get_image_embedding(image_bytes)
        
        # Search for similar products using Vector Search
        products_data = await vector_search_service.similarity_search(
            query_vector=image_embedding,
            num_results=limit
        )
        
        # Convert to ProductDetail
        products = []
        for p in products_data:
            product = ProductDetail(**p)
            product.image_url = get_image_url(int(product.product_id))
            # Similarity score comes from Vector Search
            product.similarity_score = p.get("score", 0.85)
            products.append(product)
        
        logger.info(f"Image search returned {len(products)} results")
        
        return SearchResponse(
            products=products,
            query=None,
            search_type="image",
            user_id=user_id
        )
        
    except Exception as e:
        logger.error(f"Image search error: {type(e).__name__}: {e}")
        raise HTTPException(status_code=500, detail=f"Image search failed: {str(e)}")


@router.get("/recommendations/{user_id}", response_model=SearchResponse)
async def get_recommendations(
    user_id: str,
    limit: int = 20,
    db: AsyncSession = Depends(get_async_db)
):
    """
    Hybrid personalized recommendations:
    - Vector similarity using user embeddings (60%)
    - Rule-based filtering by preferences (40%)
    """
    repo = LakebaseRepository(db)

    # Load persona to get preferences
    from routes.v1.users import load_personas

    personas = load_personas()
    persona = next((p for p in personas if p["user_id"] == user_id), None)

    if not persona:
        raise HTTPException(status_code=404, detail=f"User {user_id} not found")

    logger.info(f"Getting recommendations for user {user_id} - {persona.get('name', 'Unknown')}")
    logger.info(f"Persona preferences: categories={persona.get('preferred_categories')}, colors={persona.get('color_prefs')}")

    try:
        # Try to get user embedding from user_style_features table
        user_features = await repo.get_user_style_features(user_id)
        
        if user_features and user_features.get("user_embedding"):
            # Use Vector Search with user embedding
            from services.vector_search_service import vector_search_service
            
            user_embedding = np.array(user_features["user_embedding"], dtype=np.float32)
            
            # Build price filters for Vector Search
            min_price = persona["p25_price"] * 0.8
            max_price = persona["p75_price"] * 1.2
            
            # Vector Search with filters
            products_data = await vector_search_service.similarity_search(
                query_vector=user_embedding,
                num_results=limit * 2,  # Get more for additional filtering
                filters={"price >= ": min_price, "price <= ": max_price}
            )
            
            logger.info(f"Vector Search returned {len(products_data)} products")
            
        else:
            # Fallback to rule-based if no user embedding
            logger.warning(f"No user embedding found for {user_id}, using rule-based recommendations")
            raise Exception("No user embedding - use fallback")
            
    except Exception as e:
        logger.warning(f"Vector Search failed, using rule-based fallback: {e}")
        
        # Fallback: Rule-based recommendations
        filters = {}
        filters["min_price"] = persona["p25_price"] * 0.8
        filters["max_price"] = persona["p75_price"] * 1.2
        
        # ✅ ADD: Filter by preferred master_category
        if persona.get("preferred_categories"):
            filters["master_category"] = persona["preferred_categories"][0]
            logger.info(f"Filtering by category: {filters['master_category']}")
        
        products_data = await repo.get_products(
            limit=limit * 3,
            filters=filters
        )

    # Normalize preferred colors to Title Case for matching
    preferred_colors = set(c.title() for c in persona["color_prefs"])
    logger.info(f"Normalized color preferences: {preferred_colors}")
    
    filtered_products = []

    for p in products_data:
        # Normalize product color to Title Case
        product_color = (p["base_color"] or "").title()
        color_match = product_color in preferred_colors
        
        # Check category match
        category_match = p.get("master_category") in persona.get("preferred_categories", [])

        product = ProductDetail(**p)
        product.image_url = get_image_url(int(product.product_id))

        # Calculate hybrid score
        # If from Vector Search, use that score (60%) + rules (40%)
        # If rule-based only, use rules (100%)
        vector_score = p.get("score", 0.5)  # From Vector Search or default
        rule_score = 0.0
        
        # Category match bonus
        if category_match:
            rule_score += 0.3
        
        # Color match bonus
        if color_match:
            rule_score += 0.4
        
        # Price match bonus
        price_diff = abs(p["price"] - persona["avg_price"])
        price_range = persona["max_price"] - persona["min_price"]
        if price_range > 0:
            price_score = 1 - (price_diff / price_range)
            rule_score += 0.3 * max(0, price_score)
        
        # Hybrid score: 60% vector + 40% rules
        if "score" in p:  # Has vector similarity
            product.similarity_score = 0.6 * vector_score + 0.4 * rule_score
        else:  # Rule-based only
            product.similarity_score = rule_score

        # Add personalization reasons
        reasons = []
        if category_match:
            reasons.append(f"Matches your interest in {p['master_category']}")
        if color_match:
            reasons.append(f"Matches your preference for {product_color} items")
        if persona["min_price"] <= p["price"] <= persona["max_price"]:
            reasons.append(f"Within your typical price range (${persona['min_price']:.0f}-${persona['max_price']:.0f})")
        if "score" in p and p["score"] > 0.8:
            reasons.append("Similar to items you've liked before")

        if reasons:
            product.personalization_reason = " • ".join(reasons)

        filtered_products.append(product)

    # Sort by hybrid score and limit
    filtered_products.sort(key=lambda x: x.similarity_score or 0, reverse=True)
    products = filtered_products[:limit]
    
    logger.info(f"Returning {len(products)} personalized recommendations (avg score: {np.mean([p.similarity_score for p in products]):.2f})")

    return SearchResponse(
        products=products,
        query=None,
        search_type="personalized",
        user_id=user_id
    )

In [0]:
# Add these to your Settings class in core/config.py

# CLIP Model Serving
CLIP_ENDPOINT: str = "clip-image-encoder"
CLIP_EMBEDDING_DIM: int = 512

# Vector Search
VS_ENDPOINT_NAME: str = "fashion_vector_search"
VS_ENDPOINT_ID: str = "4d329fc8-1924-4131-ace8-14b542f8c14b"
VS_INDEX_NAME: str = "main.fashion_demo.product_embeddings_index"

# Workspace (already exists, just verify)
WORKSPACE_HOST: str = os.getenv("DATABRICKS_HOST", "")

# Add these dependencies to requirements.txt

# Vector Search
databricks-vector-search>=0.22

# HTTP client for async requests
aiohttp>=3.9.0

# Already have these (verify):
# numpy>=1.24.0
# databricks-sdk>=0.18.0

# Implementation Steps

## Step 1: Create services/ directory
```bash
cd fashion-ecom-site
mkdir -p services
touch services/__init__.py
```

## Step 2: Copy service modules
- **Cell 1** → `services/clip_service.py`
- **Cell 2** → `services/vector_search_service.py`

## Step 3: Update existing files
- **Cell 3** → Replace `routes/v1/search.py`
- **Cell 4** → Add to `core/config.py` (Settings class)
- **Cell 5** → Add to `requirements.txt`

## Step 4: Install dependencies
```bash
pip install databricks-vector-search>=0.22 aiohttp>=3.9.0
```

## Step 5: Fix personas.json (CRITICAL!)
The file is currently empty. Copy the corrected JSON from the other notebook.

## Step 6: Redeploy app

---

## What You'll Get

### Text Search (Semantic)
- Query: "red summer dress"
- Uses CLIP to understand meaning
- Finds "Scarlet Sundress", "Coral Maxi Dress" (semantic matches!)
- Ranked by vector similarity

### Image Search (Visual)
- Upload photo of a dress
- CLIP generates image embedding
- Finds visually similar products
- Works even if colors/styles differ slightly

### Recommendations (Hybrid)
- **60%** Vector similarity (user embedding vs product embeddings)
- **40%** Rule-based (category, color, price preferences)
- Personalized reasons: "Similar to items you've liked before"
- Each persona gets truly different results